# Prompt Template Prompting

Prompt Template Prompting refers to a technique where predefined templates are used to construct effective prompts that guide large language models (LLMs) to generate responses tailored to specific use cases. The templates typically contain static text combined with dynamic input variables, allowing for consistent, reusable, and customizable prompts.

Prompt templates are widely used across various domains, such as:
* **Question Generation**: Templates can generate quiz questions by filling in variables related to topics.
* **Text Summarization**: Static instructions combined with variable documents or inputs allow flexible summarization.
* **Coding Assistance**: Dynamic prompts help LLMs generate code snippets for different programming tasks.

## References:

* (OpenAI Documentation for Prompt Engineering)[https://platform.openai.com/docs/guides/prompt-engineering]

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fprompt_template.ipynb)


In [2]:
##
## PROMPT TEMPLATE PROMPTING
##
import json
from prompt.prompt_refiner import iterative_prompt_refinement, format_final_prompt
from prompt.utils import print_step
from config._pipeline import create_payload, model_req

# Original query
MESSAGE = "What causes an tidal bore?"
print_step("ORIGINAL QUERY", MESSAGE, is_json=False)

# Get recursively refined template configuration with error handling
try:
    # Get recursively refined template configuration
    refined_config = iterative_prompt_refinement(MESSAGE, min_iterations=3)
    print_step("REFINED CONFIGURATION", refined_config)
    
    # Format the prompt using the helper function
    PROMPT = format_final_prompt(refined_config, MESSAGE)
    print_step("GENERATED PROMPT", PROMPT, is_json=False)
    
    # Extract parameters (which are now validated in the refiner)
    parameters = refined_config.get("parameters", {})
    
    # Configure model with optimized parameters
    payload = create_payload(
        target="open-webui",
        model="llama3.2:latest", 
        prompt=PROMPT,
        **parameters
    )
    print_step("MODEL PAYLOAD", payload)
    
    # Get final response
    time, response = model_req(request_payload=payload)
    print_step("MODEL RESPONSE", response, is_json=False)
    if time > 0:
        print(f"Time taken: {time:.2f}s")
    else:
        print("ERROR: Request failed")

except Exception as e:
    error_message = f"An error occurred: {str(e)}\n\nTraceback:"
    print_step("ERROR", error_message, is_json=False)
    import traceback
    traceback.print_exc()


  ORIGINAL QUERY

What causes an tidal bore?

{'model': 'llama3.2:latest', 'messages': [{'role': 'user', 'content': 'You will analyze a user query and provide a JSON response. Your response must ONLY contain valid JSON with no commentary before or after. The JSON must be on a single line with no line breaks within values. All strings must use double quotes. The JSON must be parseable by json.loads().\n\n\n        Evaluate this candidate prompt: "What causes an tidal bore?"\n        \n        1. Rate the current prompt quality from 0.0 to 1.0\n        2. Provide an improved version even if quality is high\n        3. Determine the optimal role and parameters for this query\n        \n        Return your analysis in JSON format:\n        {\n            "quality_score": [score between 0-1],\n            "improved_prompt": "[refined prompt]",\n            "role": "[appropriate expert role]",\n            "template": "[prompt template with {query} placeholder]",\n            "parameters": 